In [9]:
!pip install opencv-python

In [10]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os

In [11]:
PATH = os.path.join(os.path.abspath("."), "frames", "train")

In [12]:
stable = cv2.imread(os.path.join(PATH, "stable", "stable_1.png"))
malfunction = cv2.imread(os.path.join(PATH, "malfunction", "malfunction_1.png"))

In [13]:
# copy of malfunction image that'll need later.
original = malfunction.copy()
# we subtract the malfunction image from the stable one.
image = cv2.absdiff(malfunction, stable)
# we produce a kernel (a 5x5 matrix filled with value 1)
kernel = np.ones((5,5), np.uint8)
# we erode the image to get rid of the contours left behind from the global noise.
image = cv2.erode(image, kernel, iterations=1)

In [14]:
# map the image's colormap to hsv.
image = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
# we produce a mask by selecting non-black regions of the eroded image.
lower_white = np.array([10,10,10])
upper_white = np.array([255,255,255])
mask = cv2.inRange(image, lower_white, upper_white)
# find the contours of the mask
cnts = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]

In [15]:
offset = 5

def bbArea(c):
    _,_,w,h = cv2.boundingRect(c)
    return w*h

# Since we have general noise it's best to skip over small captured
# malfunction regions. This is a more suitable solution than trying
# to erode the image for 2 iterations since it makes us lose parts of
# the targeted region. An idea is to use percentiles.
percentile = np.percentile([bbArea(c) for c in cnts], 60)

for c in cnts:
    x,y,w,h = cv2.boundingRect(c)
    if w * h > percentile:
        cv2.rectangle(original, (x - offset, y - offset), (x + w + offset, y + h + offset), (255,0,0), 2)

cv2.imshow("Stable", stable)
cv2.imshow("Malfunction", malfunction)
cv2.imshow("Mask", mask)
cv2.imshow("Malfunction Detection", original)

In [16]:
cv2.waitKey(0)
cv2.destroyAllWindows()